Install packages

Add ifind

In [1]:
import os
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-17-openjdk-amd64/'
os.environ['JVM_PATH'] = os.environ['JAVA_HOME'] + 'lib/server/libjvm.so'

In [2]:
!export PYTHONPATH="${PYTHONPATH}:./ifind"b

Prepare topics

Run simulations

In [10]:
!rm /workspace/sims/output/*

In [8]:
!cd workspace/simiir/ && python run_simiir.py ../sims/simulation_wapo_bm25.xml

PyTerrier 0.9.2 has loaded Terrier 5.7 (built by craigm on 2022-11-10 18:30) and terrier-helper 0.0.7

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.
SIMULATION 'sim_wapo_bm25-41-fixeddepthuser10_doc2query_nofeedback'
  Simulation Configuration:
    Topic: 41
    Search Interface: PyterrierWebSearchInterface
        index_dir: 
        corpus: wapo

  User Configuration (fixeddepthuser10_doc2query_nofeedback):
    Query Generator: Doc2QueryGeneratorWapo
        stopword_file: /workspace/data/stopword-list.txt
        query_file: /workspace/data/wapo/title_queries
        user: 1
        use_relevant: False
        corpus: wapo
    Snippet Classifier: StochasticInformedTrecTextClassifier
        qrel_file: /workspace/data/wapo/wapo_qrels
        rprob: 1.0
        nprob: 0.3
        data_handler: 0
    Document Classifier: StochasticInformedTrecTextClassifier
        qrel_file: /workspace/data/wapo/wapo_qrels
        rprob: 1.0
        nprob: 0.

In [1]:
!cd workspace/simiir/ && python run_simiir.py ../sims/simulation_wapo_monoT5.xml

PyTerrier 0.9.2 has loaded Terrier 5.7 (built by craigm on 2022-11-10 18:30) and terrier-helper 0.0.7

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.
SIMULATION 'sim_wapo_monoT5-41-fixeddepthuser10_doc2query_nofeedback'
  Simulation Configuration:
    Topic: 41
    Search Interface: PyterrierWebSearchInterface
        index_dir: 
        corpus: wapo
        neural: _monot5

  User Configuration (fixeddepthuser10_doc2query_nofeedback):
    Query Generator: Doc2QueryGeneratorWapo
        stopword_file: /workspace/data/stopword-list.txt
        query_file: /workspace/data/wapo/title_queries
        user: 1
        use_relevant: False
        corpus: wapo
    Snippet Classifier: StochasticInformedTrecTextClassifier
        qrel_file: /workspace/data/wapo/wapo_qrels
        rprob: 1.0
        nprob: 0.3
        data_handler: 0
    Document Classifier: StochasticInformedTrecTextClassifier
        qrel_file: /workspace/data/wapo/wapo_qrels
        rp

# Evaluations

## Modalities

### sDCG 

## Browsing behavior

### sDCG